In [26]:
import os
import pandas as pd
from collections import defaultdict

In [64]:



'''
DESCRIPTION-------
Example code for reading NVivo coded content from an NVivo extract into a pandas dataframe. 
Here we read in two files, one containing an extract of all NVivo coding except errors (everything_but_errors_12-22-20.xls), and 
another containing only the error coding (errors_12-22-20.xls). The code itself is trivial and the important points are contained in the NOTES.
NOTES-------
1) dependencies: the df.read_excel() in pandas uses the xlrd package internally, so you need to pip or conda install xlrd
2) need to export the NVivo extract as an .xls file 
  - NOT .xlsx, because for some reason xlrd now only supports the older .xls file type
  - NOT .csv because some cell values (description, coded text, maybe others) can contain commas and it is a hassle to work around this
3) df.read_excel() cannot read the .xls file as directly exported from NVivo (NVivo's exported .xls file is malformed according to .read_excel()'s requirements)
So as a workaround, you can "repair" NVivo's .xls file by doing the following (but the .xls files contained in ohtap/NVivo_output/data are already repaired, so 
no need to do this step unless you are working with .xls files you exported yourself from NVivo):
   -open up the .xls file that was exported from NVivo in some spreadsheet software (I used mac's Numbers application, but Google Sheets or Excel would probably work too)
   -re-export the opened file as another .xls file (which will now be well-formed for .read_excel()). In the Numbers application, what you do precisely is
     + navigate to File -> Export To -> Excel...
     + under "Advanced Options" change file type to .xls
     + in the Excel Worksheets field, select "One Per Table" and nothing else (in particular, unselect "Include a summary worksheet" if that was selected by default")
     + click next and save the repaired file where you want
'''



#Set up directory structure however you want. Here the 'repaired' (as described above) .xls files are assumed to reside in the directory ./data/nvivo_content
NVIVO_CONTENT_FP = os.path.join("data", "nvivo_content")
NOT_ERRORS_FP = os.path.join(NVIVO_CONTENT_FP, "everything_but_errors_12-22-20.xls")
ERRORS_FP = os.path.join(NVIVO_CONTENT_FP, "errors_12-22-20.xls")

METADATA_FP = os.path.join("data", "metadata-files")
INTERVIEWEES_METADATA_FP = os.path.join(METADATA_FP, "OHTAP_metadata - Interviewees.csv")

everything_df = pd.read_excel(NOT_ERRORS_FP)
errors_df = pd.read_excel(ERRORS_FP)
ee_metadata = pd.read_csv(INTERVIEWEES_METADATA_FP)


In [11]:
#inspect the string handles for all the different coding types
all_coding_types = everything_df["Hierarchical Name"].unique()
all_coding_types

array(['Nodes\\\\Event (Extent)', 'Nodes\\\\Event (Extent)\\Needs Review',
       'Nodes\\\\Event (Initiating Question)',
       'Nodes\\\\Event (Initiating Question)\\Needs Review',
       'Nodes\\\\Narrative Type\\Needs Review',
       'Nodes\\\\Narrative Type\\Observations and Cultural References',
       'Nodes\\\\Narrative Type\\Personal Experience',
       'Nodes\\\\Narrative Type\\Personal Experience\\Negative Response',
       'Nodes\\\\Narrative Type\\Policy; Politics; Legal',
       'Nodes\\\\Narrative Type\\Second-hand Account',
       'Nodes\\\\Topic\\Interracial Event',
       'Nodes\\\\Topic\\Needs Review',
       'Nodes\\\\Topic\\Sexual assault or rape',
       'Nodes\\\\Topic\\Sexual harassment'], dtype=object)

In [69]:
#set codes equal to the set of codes you want to count
#codes = {'Nodes\\\\Topic\\Sexual assault or rape', 'Nodes\\\\Topic\\Sexual harassment'}
codes = {'Nodes\\\\Topic\\Sexual assault or rape'}

In [17]:
everything_df["Hierarchical Name"].value_counts()

Nodes\\Event (Extent)                                          1231
Nodes\\Event (Initiating Question)                             1075
Nodes\\Topic\Sexual assault or rape                             758
Nodes\\Narrative Type\Policy; Politics; Legal                   504
Nodes\\Narrative Type\Observations and Cultural References      386
Nodes\\Narrative Type\Second-hand Account                       244
Nodes\\Narrative Type\Personal Experience                       242
Nodes\\Topic\Interracial Event                                  223
Nodes\\Topic\Sexual harassment                                  202
Nodes\\Narrative Type\Personal Experience\Negative Response      45
Nodes\\Narrative Type\Needs Review                               27
Nodes\\Event (Extent)\Needs Review                               23
Nodes\\Topic\Needs Review                                        13
Nodes\\Event (Initiating Question)\Needs Review                   9
Name: Hierarchical Name, dtype: int64

In [73]:
#get everything coded as rape (i.e. where the Hierarchical Name field is 'Nodes\\\\Topic\\Sexual assault or rape')
rape_events = everything_df[ everything_df['Hierarchical Name'].isin(codes) ]

In [74]:
#get counts of rape event by collection
rape_events["Folder Location"].value_counts()

Files\\HM                      436
Files\\SCVF                    147
Files\\UNCTWMS                  91
Files\\BWOH                     26
Files\\SCAARJ                   15
Files\\SCAP                     10
Files\\UNCSW                     8
Files\\SCAL                      8
Files\\SHSA                      5
Files\\ROHA                      5
Files\\SHSF                      2
Files\\Missing OSS and SOOH      2
Files\\BWSP                      2
Files\\UNCGAS                    1
Name: Folder Location, dtype: int64

In [76]:
#get counts of interviews that have at least one rape event, by collection
list_of_interviews_with_rape_event = list(rape_events["Hierarchical Name.1"].unique())

counts_by_collection = defaultdict(lambda:0)
for interview in list_of_interviews_with_rape_event:
    
    collection = interview.split('\\\\')[1]
    counts_by_collection[collection] += 1

In [78]:
counts_by_collection

defaultdict(<function __main__.<lambda>()>,
            {'BWOH': 18,
             'BWSP': 2,
             'HM': 161,
             'Missing OSS and SOOH': 2,
             'ROHA': 5,
             'SCAARJ': 4,
             'SCAL': 6,
             'SCAP': 8,
             'SCVF': 35,
             'SHSA': 2,
             'SHSF': 2,
             'UNCGAS': 1,
             'UNCSW': 7,
             'UNCTWMS': 34})

In [79]:
'''
#manual counts of number of interviews in each collection, followed by description of how count was arrived at
#TODO: clean up organization of transcripts on google drive so we know how many
#interviews are in each collection and can easily get .txt files for them
total_interviews_per_collection = {
    #'BWOH': 18,
             #'BWSP': 2,
             'HM': 2687, #number of .txt files in download of HM transcripts from Google drive
             #'Missing OSS and SOOH': 2,
             #'ROHA': 5,
             #'SCAARJ': 4,
             #'SCAL': 6,
             #'SCAP': 8,
             'SCVF': 247, # UNCERTAIN, just comes from adding up counts of Smith college related interviews on Collection Info spreadsheet
             #'SHSA': 2,
             #'SHSF': 2,
             #'UNCGAS': 1,
             #'UNCSW': 7,
             #'UNCTWMS': 34}
}
'''

total_interviews_per_collection = dict(ee_metadata['collection_id'].value_counts())


In [80]:
total_interviews_per_collection

{'HM': 2687,
 'OSS': 411,
 'SOOH': 303,
 'SCAP': 199,
 'UNCTWMS': 167,
 'BWSP': 164,
 'OWDB': 143,
 'RTRB': 123,
 'OCFF': 118,
 'OOHYLC': 111,
 'ROHA': 80,
 'BWOH': 72,
 'ONA': 69,
 'UNCSW': 67,
 'SCVF': 51,
 'ONA; SOOH': 49,
 'SHSF': 47,
 'WOL': 47,
 'OWHF': 36,
 'SHSA': 30,
 'UNCGAS': 30,
 'SCAL': 15,
 'SCAARJ': 14,
 'SNAI': 13,
 'ONA; OWHF; SOOH': 1,
 'OSS; SOOH': 1}

In [82]:
for key, val in counts_by_collection.items():
    if key in total_interviews_per_collection:
        n_hit = counts_by_collection[key]
        n_total = total_interviews_per_collection[key]
        proportion = n_hit / n_total 
        print("{}: {:.3f}".format(key, proportion))

BWOH: 0.250
BWSP: 0.012
HM: 0.060
ROHA: 0.062
SCAARJ: 0.286
SCAL: 0.400
SCAP: 0.040
SCVF: 0.686
SHSA: 0.067
SHSF: 0.043
UNCGAS: 0.033
UNCSW: 0.104
UNCTWMS: 0.204


**Prepare a .csv file with columns "collection_name", "interviews_with_CODES", "total_interviews", "percentage_of_collection"**

In [143]:

n_hit = pd.Series(counts_by_collection).astype(int)

In [144]:
n_total = pd.Series(total_interviews_per_collection).astype(int)

In [145]:
df = pd.concat([n_hit, n_total], axis=1).dropna().astype(int)

In [146]:
df.rename(columns={0: "interviews_with_CODES", 1: "total_interviews"}, inplace=True)

In [147]:
df["percentage_of_collection"] = df["interviews_with_CODES"]/df["total_interviews"]

In [148]:
df.reset_index(inplace=True)

In [149]:
df.rename(columns={"index": "collection_id"}, inplace=True)

In [150]:
df1 = df.rename(columns={"interviews_with_CODES": "interviews_with_rape_topic_code"})

In [151]:
df1

,collection_id,interviews_with_rape_topic_code,total_interviews,percentage_of_collection
0,BWOH,18,72,0.250000
1,BWSP,2,164,0.012195
2,HM,161,2687,0.059918
3,ROHA,5,80,0.062500
4,SCAARJ,4,14,0.285714
5,SCAL,6,15,0.400000
6,SCAP,8,199,0.040201
7,SCVF,35,51,0.686275
8,SHSA,2,30,0.066667
9,SHSF,2,47,0.042553
